<a href="https://colab.research.google.com/github/aleks-haksly/leetcode/blob/main/pandas/pandas_easy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

1179. Reformat Department Table

https://leetcode.com/problems/reformat-department-table/description/

In [ ]:
import pandas as pd

def reformat_table(department: pd.DataFrame) -> pd.DataFrame:
    all_mths = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    pvt = department.pivot_table(index='id', columns='month', values='revenue').reindex(columns = all_mths)
    pvt.columns = [_+"_Revenue" for _ in pvt.columns]
    return pvt.reset_index()

175. Combine Two Tables

https://leetcode.com/problems/combine-two-tables/description/

In [3]:
data = [[1, 'Wang', 'Allen'], [2, 'Alice', 'Bob']]
person = pd.DataFrame(data, columns=['personId', 'firstName', 'lastName']).astype({'personId':'Int64', 'firstName':'object', 'lastName':'object'})
data = [[1, 2, 'New York City', 'New York'], [2, 3, 'Leetcode', 'California']]
address = pd.DataFrame(data, columns=['addressId', 'personId', 'city', 'state']).astype({'addressId':'Int64', 'personId':'Int64', 'city':'object', 'state':'object'})

```sql
SELECT
  p.firstName,
  p.lastName,
  a.city,
  a.state
FROM person p
LEFT JOIN Address a
ON p.personId = a.personId

```

In [6]:
person.merge(address, how='left', on='personId')[['firstName', 'lastName', 'city','state' ]]

,firstName,lastName,city,state
0,Wang,Allen,NaN,NaN
1,Alice,Bob,New York City,New York


181. Employees Earning More Than Their Managers

https://leetcode.com/problems/employees-earning-more-than-their-managers/description/

In [25]:
data = [[1, 'Joe', 70000, 3], [2, 'Henry', 80000, 4], [3, 'Sam', 60000, None], [4, 'Max', 90000, None]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'managerId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'managerId':'Int64'})

```sql
SELECT e2.name as  Employee FROM Employee e1
INNER JOIN Employee e2 on e1.id = e2.managerId
WHERE e1.salary < e2.salary
```


In [26]:
def find_employees(employee: pd.DataFrame) -> pd.DataFrame:
    merged = employee.merge(employee, how='inner', left_on='id', right_on='managerId')
    merged = merged[merged.salary_x < merged.salary_y][['name_y']].rename(columns={'name_y':'Employee'})
    return merged

182. Duplicate Emails

https://leetcode.com/problems/duplicate-emails/description/

```sql
SELECT email as "Email" FROM Person
GROUP BY email
HAVING count(email) > 1
```

In [28]:
data = [[1, 'a@b.com'], [2, 'c@d.com'], [3, 'a@b.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'Int64', 'email':'object'})

In [51]:
def duplicate_emails(person: pd.DataFrame) -> pd.DataFrame:
    return person.groupby('email', as_index=False).count().query("id > 1", engine="python")[['email']]

183. Customers Who Never Order

https://leetcode.com/problems/customers-who-never-order/description/

```sql
SELECT name as "Customers" FROM Customers c WHERE c.id NOT IN (SELECT customerId FROM Orders )
```

In [52]:
data = [[1, 'Joe'], [2, 'Henry'], [3, 'Sam'], [4, 'Max']]
customers = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})
data = [[1, 3], [2, 1]]
orders = pd.DataFrame(data, columns=['id', 'customerId']).astype({'id':'Int64', 'customerId':'Int64'})

In [64]:
def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    return customers[~customers['id']\
                     .isin(orders.customerId)][['name']]\
                     .rename({'name': 'Customers'}, axis=1)

196. Delete Duplicate Emails

https://leetcode.com/problems/delete-duplicate-emails/description/

```sql
DELETE FROM Person WHERE id in (
SELECT p1.id FROM Person p1
JOIN Person p2 ON p1.email = p2.email and p1.id > p2.id)
```


In [67]:
data = [[1, 'john@example.com'], [2, 'bob@example.com'], [3, 'john@example.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'int64', 'email':'object'})

In [81]:
def delete_duplicate_emails(person: pd.DataFrame) -> None:
    person.sort_values(by='id',ascending=True,inplace=True)
    person.drop_duplicates(subset='email', keep='first', inplace=True)

197. Rising Temperature

https://leetcode.com/problems/rising-temperature/description/

```sql

SELECT id as Id FROM (
SELECT *,
lag(temperature, 1) OVER (ORDER BY recordDate) as tlag,
lag(recordDate , 1) OVER (ORDER BY recordDate) as dlag
FROM Weather) t
WHERE temperature - tlag > 0 and recordDate - dlag = 1
```

In [94]:
data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25], [3, '2015-01-03', 20], [4, '2015-01-04', 30]]
weather = pd.DataFrame(data, columns=['id', 'recordDate', 'temperature']).astype({'id':'Int64', 'recordDate':'datetime64[ns]', 'temperature':'Int64'})

In [117]:
def rising_temperature(weather: pd.DataFrame) -> pd.DataFrame:
    weather = weather.sort_values(by='recordDate')
    df = weather[(weather['temperature'].diff()>0) & (weather['recordDate'].diff().dt.days == 1)]
    return df[['id']]

511. Game Play Analysis I

https://leetcode.com/problems/game-play-analysis-i/description/

```sql
SELECT player_id,
min(event_date) as first_login
FROM Activity
GROUP BY player_id
```

In [125]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-05-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})

In [126]:
activity

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-05-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [133]:
activity.groupby('player_id')['event_date'].min().reset_index().rename({'event_date':'first_login'}, axis =1)

,player_id,first_login
0,1,2016-03-01
1,2,2017-06-25
2,3,2016-03-02


577. Employee Bonus

https://leetcode.com/problems/employee-bonus/description/

```sql
SELECT
e.name,
b.bonus
FROM Employee e
LEFT JOIN Bonus b on b.empId = e.empId
WHERE COALESCE(b.bonus , 0) < 1000
```

```
DataFrame.eq
Compare DataFrames for equality elementwise.

DataFrame.ne
Compare DataFrames for inequality elementwise.

DataFrame.le
Compare DataFrames for less than inequality or equality elementwise.

DataFrame.lt
Compare DataFrames for strictly less than inequality elementwise.

DataFrame.ge
Compare DataFrames for greater than inequality or equality elementwise.

DataFrame.gt
Compare DataFrames for strictly greater than inequality elementwise.

```

In [134]:
data = [[3, 'Brad', None, 4000], [1, 'John', 3, 1000], [2, 'Dan', 3, 2000], [4, 'Thomas', 3, 4000]]
employee = pd.DataFrame(data, columns=['empId', 'name', 'supervisor', 'salary']).astype({'empId':'Int64', 'name':'object', 'supervisor':'Int64', 'salary':'Int64'})
data = [[2, 500], [4, 2000]]
bonus = pd.DataFrame(data, columns=['empId', 'bonus']).astype({'empId':'Int64', 'bonus':'Int64'})

In [143]:
def employee_bonus(employee: pd.DataFrame, bonus: pd.DataFrame) -> pd.DataFrame:
    merged = employee.merge(bonus, how='left', on='empId')
    return merged[merged['bonus'].lt(1000) | merged['bonus'].isnull()][['name', 'bonus']]

,name,bonus
0,Brad,<NA>
1,John,<NA>
2,Dan,500


584. Find Customer Referee

https://leetcode.com/problems/find-customer-referee/description/

```sql
SELECT name FROM Customer
WHERE referee_id <> 2 OR referee_id IS NULL
```

In [144]:
data = [[1, 'Will', None], [2, 'Jane', None], [3, 'Alex', 2], [4, 'Bill', None], [5, 'Zack', 1], [6, 'Mark', 2]]
customer = pd.DataFrame(data, columns=['id', 'name', 'referee_id']).astype({'id':'Int64', 'name':'object', 'referee_id':'Int64'})

In [167]:
def find_customer_referee(customer: pd.DataFrame) -> pd.DataFrame:
    return customer[customer['referee_id'].ne(2) | customer['referee_id'].isna()][['name']]

586. Customer Placing the Largest Number of Orders

https://leetcode.com/problems/customer-placing-the-largest-number-of-orders/description/

```sql
SELECT customer_number FROM Orders
GROUP BY customer_number
ORDER BY count(order_number) DESC
LIMIT 1
```

In [150]:
data = [[1, 1], [2, 2], [3, 3], [4, 3]]
orders = pd.DataFrame(data, columns=['order_number', 'customer_number']).astype({'order_number':'Int64', 'customer_number':'Int64'})

In [168]:
def largest_orders(orders: pd.DataFrame) -> pd.DataFrame:
    return orders.groupby('customer_number', as_index=False)\
                 .count()\
                 .sort_values('order_number', ascending=False)\
                 .head(1)[['customer_number']]

595. Big Countries

https://leetcode.com/problems/big-countries/

```sql
SELECT
    w.name,
    w.population,
    w.area
FROM World w
    WHERE area >= 3000000 OR population >= 25000000
```

In [169]:
data = [['Afghanistan', 'Asia', 652230, 25500100, 20343000000], ['Albania', 'Europe', 28748, 2831741, 12960000000], ['Algeria', 'Africa', 2381741, 37100000, 188681000000], ['Andorra', 'Europe', 468, 78115, 3712000000], ['Angola', 'Africa', 1246700, 20609294, 100990000000]]
world = pd.DataFrame(data, columns=['name', 'continent', 'area', 'population', 'gdp']).astype({'name':'object', 'continent':'object', 'area':'Int64', 'population':'Int64', 'gdp':'Int64'})

In [171]:
def big_countries(world: pd.DataFrame) -> pd.DataFrame:
    return world[world.area.ge(3000000) | world.population.ge(25000000)][['name', 'population', 'area']]

596. Classes More Than 5 Students

https://leetcode.com/problems/classes-more-than-5-students/

```sql
SELECT class FROM Courses c
GROUP BY class
HAVING count(student) >= 5
```

In [172]:
data = [['A', 'Math'], ['B', 'English'], ['C', 'Math'], ['D', 'Biology'], ['E', 'Math'], ['F', 'Computer'], ['G', 'Math'], ['H', 'Math'], ['I', 'Math']]
courses = pd.DataFrame(data, columns=['student', 'class']).astype({'student':'object', 'class':'object'})

In [181]:
def find_classes(courses: pd.DataFrame) -> pd.DataFrame:
    return courses.groupby('class', as_index=False)['student'].count().query("student >= 5")[['class']]

607. Sales Person

https://leetcode.com/problems/sales-person/description/

```sql
SELECT sp.name FROM SalesPerson sp
WHERE sp.sales_id  NOT IN (
SELECT o.sales_id FROM Company c
INNER JOIN Orders o ON o.com_id = c.com_id
WHERE c.name = 'RED')
```

In [182]:
data = [[1, 'John', 100000, 6, '4/1/2006'], [2, 'Amy', 12000, 5, '5/1/2010'], [3, 'Mark', 65000, 12, '12/25/2008'], [4, 'Pam', 25000, 25, '1/1/2005'], [5, 'Alex', 5000, 10, '2/3/2007']]
sales_person = pd.DataFrame(data, columns=['sales_id', 'name', 'salary', 'commission_rate', 'hire_date']).astype({'sales_id':'Int64', 'name':'object', 'salary':'Int64', 'commission_rate':'Int64', 'hire_date':'datetime64[ns]'})
data = [[1, 'RED', 'Boston'], [2, 'ORANGE', 'New York'], [3, 'YELLOW', 'Boston'], [4, 'GREEN', 'Austin']]
company = pd.DataFrame(data, columns=['com_id', 'name', 'city']).astype({'com_id':'Int64', 'name':'object', 'city':'object'})
data = [[1, '1/1/2014', 3, 4, 10000], [2, '2/1/2014', 4, 5, 5000], [3, '3/1/2014', 1, 1, 50000], [4, '4/1/2014', 1, 4, 25000]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'com_id', 'sales_id', 'amount']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'com_id':'Int64', 'sales_id':'Int64', 'amount':'Int64'})


In [189]:
company.merge(orders, how='left', on='com_id').query("name == 'RED'").sales_id

,sales_id
0,1
1,4


In [191]:
def sales_person(sales_person: pd.DataFrame, company: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    filtered = company.merge(orders, how='left', on='com_id').query("name == 'RED'").sales_id
    return sales_person[~sales_person['sales_id'].isin(filtered)][['name']]

,name
1,Amy
2,Mark
4,Alex


610. Triangle Judgement

https://leetcode.com/problems/triangle-judgement/description/

```sql
SELECT
*,
(CASE WHEN (x + y > z AND z + y > x AND x + z > y ) THEN 'Yes' ELSE 'No' END) AS triangle
FROM Triangle
```

In [212]:
data = [[13, 15, 30], [10, 20, 15]]
triangle = pd.DataFrame(data, columns=['x', 'y', 'z']).astype({'x':'Int64', 'y':'Int64', 'z':'Int64'})

In [214]:
def triangle_judgement(triangle: pd.DataFrame) -> pd.DataFrame:
    condition = ((triangle['x'] + triangle['y'] > triangle['z']) &
                (triangle['x'] + triangle['z'] > triangle['y']) &
                (triangle['y'] + triangle['z'] > triangle['x']))
    triangle['triangle'] = condition.map({False: 'No', True: 'Yes'})
    return triangle

619. Biggest Single Number

https://leetcode.com/problems/biggest-single-number/description/

```sql
SELECT max(num) as num  FROM (
SELECT num FROM MyNumbers mn
GROUP BY num
HAVING count(*) = 1
ORDER BY num DESC
LIMIT 1) t
```


In [244]:
data = [[8], [8], [3], [3], [1], [4], [5], [6]]
my_numbers = pd.DataFrame(data, columns=['num']).astype({'num':'Int64'})

In [245]:
def biggest_single_number(my_numbers: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame([my_numbers.value_counts().reset_index().query("count == 1").num.max()],
                        index=[0],
                        columns=['num']
                        )

620. Not Boring Movies

https://leetcode.com/problems/not-boring-movies/description/

```sql
SELECT * FROM Cinema
WHERE (description <> 'boring' OR description is NULL) AND id % 2 = 1
ORDER BY rating  DESC
```

In [253]:
data = [[1, 'War', 'great 3D', 8.9], [2, 'Science', 'fiction', 8.5], [3, 'irish', 'boring', 6.2], [4, 'Ice song', 'Fantacy', 8.6], [5, 'House card', 'Interesting', 9.1]]
cinema = pd.DataFrame(data, columns=['id', 'movie', 'description', 'rating']).astype({'id':'Int64', 'movie':'object', 'description':'object', 'rating':'Float64'})

In [262]:
cinema[~((cinema["description"] == 'boring') | cinema.description.isna() | cinema.id % 2 == 0)]

,id,movie,description,rating
0,1,War,great 3D,8.9
2,3,irish,boring,6.2
4,5,House card,Interesting,9.1


1050. Actors and Directors Who Cooperated At Least Three Times

https://leetcode.com/problems/actors-and-directors-who-cooperated-at-least-three-times/description/

```sql
SELECT
actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING count(timestamp) > 2
```

In [3]:
data = [[1, 1, 0], [1, 1, 1], [1, 1, 2], [1, 2, 3], [1, 2, 4], [2, 1, 5], [2, 1, 6]]
actor_director = pd.DataFrame(data, columns=['actor_id', 'director_id', 'timestamp']).astype({'actor_id':'int64', 'director_id':'int64', 'timestamp':'int64'})

In [10]:
def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    return actor_director.groupby(["actor_id", "director_id"])["timestamp"]\
                         .count()\
                         .reset_index()\
                         .query("timestamp > 2")[["actor_id", "director_id"]]

1068. Product Sales Analysis I

https://leetcode.com/problems/product-sales-analysis-i/description/

```sql
SELECT p.product_name, s.year, s.price
FROM Sales s
LEFT JOIN Product p on p.product_id  = s.product_id
```

In [11]:
data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype({'sale_id':'Int64', 'product_id':'Int64', 'year':'Int64', 'quantity':'Int64', 'price':'Int64'})
data = [[100, 'Nokia'], [200, 'Apple'], [300, 'Samsung']]
product = pd.DataFrame(data, columns=['product_id', 'product_name']).astype({'product_id':'Int64', 'product_name':'object'})

In [14]:
def sales_analysis(sales: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    return sales.merge(product, how='left', on='product_id')[["product_name", "year", "price"]]

1075. Project Employees I

https://leetcode.com/problems/project-employees-i/description/

```sql
SELECT
p.project_id,
round(avg(e.experience_years), 2) as average_years
FROM Project p
LEFT JOIN Employee e on p.employee_id = e.employee_id
GROUP BY p.project_id
```

In [15]:
data = [[1, 1], [1, 2], [1, 3], [2, 1], [2, 4]]
project = pd.DataFrame(data, columns=['project_id', 'employee_id']).astype({'project_id':'Int64', 'employee_id':'Int64'})
data = [[1, 'Khaled', 3], [2, 'Ali', 2], [3, 'John', 1], [4, 'Doe', 2]]
employee = pd.DataFrame(data, columns=['employee_id', 'name', 'experience_years']).astype({'employee_id':'Int64', 'name':'object', 'experience_years':'Int64'})

In [17]:
def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    return project.merge(employee, how='left', on='employee_id')\
                  .groupby("project_id")["experience_years"]\
                  .mean()\
                  .round(2)\
                  .rename("average_years")\
                  .reset_index()

1084. Sales Analysis III

https://leetcode.com/problems/sales-analysis-iii/description/

```sql
SELECT
p.product_id,
p.product_name
FROM Sales s
LEFT JOIN Product p ON p.product_id = s.product_id
GROUP BY p.product_id, p.product_name
HAVING min(sale_date  ) >= '2019-01-01' and max(sale_date  ) <= '2019-03-31'

```

In [18]:
data = [[1, 'S8', 1000], [2, 'G4', 800], [3, 'iPhone', 1400]]
product = pd.DataFrame(data, columns=['product_id', 'product_name', 'unit_price']).astype({'product_id':'Int64', 'product_name':'object', 'unit_price':'Int64'})
data = [[1, 1, 1, '2019-01-21', 2, 2000], [1, 2, 2, '2019-02-17', 1, 800], [2, 2, 3, '2019-06-02', 1, 800], [3, 3, 4, '2019-05-13', 2, 2800]]
sales = pd.DataFrame(data, columns=['seller_id', 'product_id', 'buyer_id', 'sale_date', 'quantity', 'price']).astype({'seller_id':'Int64', 'product_id':'Int64', 'buyer_id':'Int64', 'sale_date':'datetime64[ns]', 'quantity':'Int64', 'price':'Int64'})

In [29]:
def sales_analysis(product: pd.DataFrame, sales: pd.DataFrame) -> pd.DataFrame:
    merged = sales.merge(product, how='left', on='product_id')\
                  .groupby(["product_id", "product_name"])\
                  .agg({"sale_date": ["min", "max"]})
    merged.columns = merged.columns.droplevel()
    return merged.query("min >= '2019-01-01' & max <= '2019-03-31'")\
                 .reset_index()[["product_id","product_name"]]